### 오류 데이터 생성.
	- G2P(Grapheme to Phoneme)를 이용한 오류 데이터 생성. (g2pk 모듈 사용)
	- 자모 단위 철자 오류 데이터 생성.
	- 통번역 데이터 기반 오류 데이터 생성. -> 오탈자 리스트 구축이 필요함.
    
    - Transformer VS Language model
    

#### Data set

- 전체 데이터
    1. G2P data : 50%
    2. 자모 단위 철자 오류 데이터 : 50%
    
    -- ** 통번역 데이터 기반 오류 데이터의 경우 리스트 구축에 시간소모가 클 것으로 예상되어 우선 제외함.

In [1]:
import json
import os
import re
import random
from tqdm import tqdm
import itertools

import pandas as pd
import numpy as np

from g2pk import G2p
from levenshtein_finder import LevenshteinFinder , CharacterTokenizer , Normalizers

In [3]:
g2p = G2p()
g2p('값어치')

'갑써치'

In [2]:
path = '../data'
enko = pd.read_csv('../data/aihub_food_enko.csv')
zhko = pd.read_csv('../data/aihub_food_zhko.csv')

In [38]:
df = pd.concat([enko[['tgt']], zhko[['tgt']]], axis=0)
df.reset_index(inplace=True, drop=True)
ko = df['tgt'].tolist()

In [135]:
r_list = []
for i in range(len(df)):
    if len(re.findall('[^ㄱ-힣|\.|\s]', df['tgt'][i])) == 0:
        r_list.append(i)
only_ko = df.loc[r_list]
only_ko.reset_index(inplace=True, drop=True)
print(only_ko.head())
print(len(only_ko))

                                                 tgt
0  이 문제에 대한 새로운 접근법은 나이신과 같은 항균제를 식품 가공 표면에 흡착시키는...
1        격막 형성의 실패는 아마도 세포가 격막 합성을 시작하지 못하기 때문일 것이다.
2  지난 세기 말에 베네수엘라의 로드니우스 프롤릭서스와 브라질의 또 다른 중요한 샤가스...
3   모기 풀 스크리닝은 바이러스 활동에 대한 가장 시기적절한 지표를 제공할 가능성이 있다.
4               사립학교 아이들은 공립학교 아이들보다 아스파탐을 덜 섭취했습니다.
733822


In [31]:
random.seed(825)
np.random.seed(826)

In [136]:
# 말뭉치 단어 토크나이징(split) 
corpus = [sent.strip().split(" ") for sent in only_ko['tgt'].tolist()]

# 2차원 리스트(문장이 단어로 분리된)를 1차원으로 통합(단어 단위)
corpus = list(itertools.chain(*corpus))

# 중복된 단어 제거
corpus = list(set(corpus))
 
# 자소 단위로 쪼개서 편집거리를 구하는 알고리즘.
normalizer = Normalizers.create_normalizer(unicodedata=True)
tokenizer = CharacterTokenizer(normalizer=normalizer)
finder = LevenshteinFinder(tokenizer=tokenizer)
finder.indexing(corpus)

  0%|                                                                                                                            | 0/50000 [74:35:08<?, ?it/s]


In [137]:
# 편집거리 테스트
finder.search('뭐해')

[{'idx': 160396, 'data': '매해', 'distance': 1},
 {'idx': 322918, 'data': '무해', 'distance': 1}]

### 노이즈 데이터 생성

In [138]:
ko = only_ko['tgt'].tolist()

In [139]:
t_idx = random.sample(range(len(ko)), 100000)
error_idx = random.sample(t_idx, len(t_idx)//2)

In [151]:
df_t = only_ko.loc[t_idx]
# df_t.reset_index(inplace=True, drop=True)

In [152]:
gtp = df_t[['tgt']].loc[error_idx]
edit_dist = df_t[['tgt']].drop(error_idx)

In [153]:
gtp.reset_index(inplace=True, drop=True)
edit_dist.reset_index(inplace=True, drop=True)

In [154]:
print(len(only_ko))
print(len(gtp))
print(len(edit_dist))

733822
50000
50000


In [155]:
gtp['tgt'][0]

'증류수로 희석한 탈지유와 수프는 생산자 지침에 표시된 농도의 절반으로 증류수로 만들었다.'

In [156]:
g2p_data = [g2p(gtp['tgt'][k]) for k in tqdm(range(len(gtp)))]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [25:12<00:00, 33.06it/s]


In [157]:
print(g2p_data[0])
print(gtp['tgt'][0])

증뉴수로 히서칸 탈지유와 수프는 생산자 지치메 표시된 농도의 절바느로 증뉴수로 만드럳따.
증류수로 희석한 탈지유와 수프는 생산자 지침에 표시된 농도의 절반으로 증류수로 만들었다.


In [209]:
kor_begin = 44032
kor_end = 55203

chosung_base = 588
jungsung_base = 28

jaum_begin = 12593
jaum_end = 12622

moum_begin = 12623
moum_end = 12643

chosung_list = [ 'ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 
        'ㅅ', 'ㅆ', 'ㅇ' , 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

jungsung_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 
        'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 
        'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 
        'ㅡ', 'ㅢ', 'ㅣ', ' ']

jongsung_list = [
    ' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ',
        'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 
        'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 
        'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

jaum_list = ['ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄸ', 'ㄹ', 
              'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 
              'ㅃ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

moum_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 
              'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']

def compose(chosung, jungsung, jongsung):
    # 자음 모음 결합
    char = chr(
        kor_begin +
        chosung_base * chosung_list.index(chosung) +
        jungsung_base * jungsung_list.index(jungsung) +
        jongsung_list.index(jongsung)
    )
    return char

def decompose(c):
    # 자음 모음 분해
    if not character_is_korean(c):
        return None
    i = ord(c)
    if (jaum_begin <= i <= jaum_end):
        return (c, ' ', ' ')
    if (moum_begin <= i <= moum_end):
        return (' ', c, ' ')

    # decomposition rule
    i -= kor_begin
    cho  = i // chosung_base
    jung = ( i - cho * chosung_base ) // jungsung_base 
    jong = ( i - cho * chosung_base - jung * jungsung_base )    
    return (chosung_list[cho], jungsung_list[jung], jongsung_list[jong])

def character_is_korean(c):
    i = ord(c)
    return ((kor_begin <= i <= kor_end) or
            (jaum_begin <= i <= jaum_end) or
            (moum_begin <= i <= moum_end))


In [210]:
from time import time

In [214]:
%%time
dist_list = []
for i in range(len(edit_dist)):
    # 띄어쓰기를 기준으로 split
    split_t = edit_dist['tgt'][i].split()
    
    # sample number
    sample_num = random.randrange(1,4)
    
    # 샘플 갯수에 맞게 choose
    if len(split_t) <= sample_num:
        sample_num = sample_num - 1
        rand_t = random.sample(range(len(split_t)), sample_num)
    else:
        rand_t = random.sample(range(len(split_t)), sample_num)
    
    for n in rand_t:
        # 한글 제외 모두 제거.
        repl_text = re.sub('[^ㄱ-힣]', '', split_t[n])
        repl_text = repl_text.strip()
        repl_text = repl_text.replace(' ', '')
        
        t = random.choice(range(len(repl_text)))
        
        # 자모 분리
        decomp_text = decompose(repl_text[t])
        decomp_text = list(decomp_text)
        c = random.choice(range(len(decomp_text)))
        
        if decomp_text[c] in jongsung_list:
            if c == 0:
                decomp_text[c] = random.choice(chosung_list)
                comp = compose(decomp_text[c], decomp_text[1], decomp_text[-1])
            elif c == 2:
                decomp_text[c] = random.choice(jongsung_list)
                comp = compose(decomp_text[0], decomp_text[1], decomp_text[c])
                
        elif decomp_text[c] in jungsung_list:
            if len(decomp_text) == 2:
                decomp_text[c] = random.choice(jungsung_list)
                comp = compose(decomp_text[0], decomp_text[c])
            elif len(decomp_text) > 2:
                decomp_text[c] = random.choice(jungsung_list)
                comp = compose(decomp_text[0], decomp_text[c], decomp_text[-1])
        
        # 변환을 위한 문자열 -> list
        text_l = list(repl_text)
        text_l[t] = comp
        repl_text = ''.join(text_l)
        split_t[n] = repl_text
    dist = ' '.join(split_t)
    
    dist_list.append(dist)
    

CPU times: user 1.51 s, sys: 6.76 ms, total: 1.51 s
Wall time: 1.52 s


In [215]:
len(dist_list)

50000

In [220]:
dist_list[0]

'중국엂로 조리는 먹을 새 있는 것을 특정한 방식으로 익힌다는 것을 의미함뜰 알 수 있습니다.'

In [221]:
edit_dist['tgt'][0]

'중국어로 조리는 먹을 수 있는 것을 특정한 방식으로 익힌다는 것을 의미함을 알 수 있습니다.'

In [224]:
# g2p를 사용한 오타데이터 : g2p_data
# 자모 분리후 오타 데이터 생성 : dist_list

In [227]:
raw_gtp = gtp['tgt'].tolist()
raw_dist = edit_dist['tgt'].tolist()

In [234]:
src = g2p_data + dist_list
tgt = raw_gtp + raw_dist

In [238]:
print(src[200])
print(tgt[200])

print(src[50001])
print(tgt[50001])

농촌단체급씨긔 식푸만전괄리는 지방정부의 성과평가내용과 중요지표에 포함되어야 하며 농촌단체급씨긔 식푸만전괄리영냥은 지속쩌그로 강화해야 함니다.
농촌단체급식의 식품안전관리는 지방정부의 성과평가내용과 중요지표에 포함되어야 하며 농촌단체급식의 식품안전관리역량은 지속적으로 강화해야 합니다.
아유 시장 옹호자들과 자유지상주의자들은 옥수수 보조금이 만들어내는 시장 왜곡괃 비효율성을 오랫동안 비난해 왔다.
자유 시장 옹호자들과 자유지상주의자들은 옥수수 보조금이 만들어내는 시장 왜곡과 비효율성을 오랫동안 비난해 왔다.


In [239]:
t_df = pd.DataFrame({'src':src, 'tgt':tgt})

t_df.head()

,src,tgt
0,증뉴수로 히서칸 탈지유와 수프는 생산자 지치메 표시된 농도의 절바느로 증뉴수로 만드럳따.,증류수로 희석한 탈지유와 수프는 생산자 지침에 표시된 농도의 절반으로 증류수로 만들었다.
1,다른 한펴느로는 파괴려게 대한 함니저기 뉴연성과 빌딩 블록꽈 신호 분자가 사라인는 ...,다른 한편으로는 파괴력에 대한 합리적인 유연성과 빌딩 블록과 신호 분자가 살아있는 ...
2,식푸뮈생감도근 기술과제임과 동시에 버베 따른 행정저 검무로서 감독짜는 견실하고 유연...,식품위생감독은 기술과제임과 동시에 법에 따른 행정적 업무로서 감독자는 견실하고 유연...
3,예시메 합껴칸 자는 시 품질감독뿌서가 성급 품질감독뿌서와 품질감독껌사거멱총구게 보고...,예심에 합격한 자는 시 품질감독부서가 성급 품질감독부서와 품질감독검사검역총국에 보고...
4,케이크 가루는 기포 분포가 매우 규닐하며 베이킹 후에도 월래 구조가 여전히 유지되고...,케이크 가루는 기포 분포가 매우 균일하며 베이킹 후에도 원래 구조가 여전히 유지되고...


In [241]:
t_df.to_csv('../data/korean_correct_train_data_100000.csv', encoding='utf-8-sig', index=False)